In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys, os, pathlib
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/pkgconfig'

import numpy as np
import pandas as pd
import torch
import nibabel as nib

sys.path.append('../..')
import project

torch.cuda.is_available()

True

In [4]:
%autoreload
data_root = pathlib.Path('/ocean/projects/asc170022p/mtragoza/lung-project/data/COPDGene')
data_file = data_root / 'sample1000_2025-07-22.csv'

# load sampled data file
df_sampled = pd.read_csv(data_file, sep='\t', low_memory=False)
df_sampled

,sid,ccenter,kernel,Emphysema,pctEmph,pctEmph_Slicer,FEV1pp_utah,FVCpp_utah,FEV1_FVC_utah,finalGold,catEmph
0,16514P,TEM,STD,0,0.3373,0.3147,77.6,88.9,0.71,-1.0,normal
1,20748Q,UAB,STD,0,3.6045,3.4085,73.2,75.8,0.75,-1.0,normal
2,11007Z,USD,STD,0,0.3126,0.2867,66.7,108.0,0.47,2.0,normal
3,14771Z,HPR,STD,0,0.3160,0.2948,94.1,97.0,0.73,0.0,normal
4,13651K,UIA,STD,0,2.0807,1.9551,85.1,88.0,0.75,0.0,normal
...,...,...,...,...,...,...,...,...,...,...,...
995,20519B,DUK,STD,1,30.1942,29.6440,19.9,69.2,0.22,4.0,severe
996,12294H,UAB,STD,1,40.3908,39.5845,43.3,86.0,0.37,3.0,severe
997,23123R,TEM,STD,0,23.6838,22.7617,42.8,72.7,0.45,3.0,severe
998,16546C,UIA,STD,1,42.6116,41.8307,58.6,99.6,0.43,2.0,severe


In [7]:
dataset = project.copdgene.COPDGeneDataset(df_sampled, data_root)
dataset

In [8]:
row, visit = dataset[0]
row

sid               16514P
ccenter              TEM
kernel               STD
Emphysema              0
pctEmph           0.3373
pctEmph_Slicer    0.3147
FEV1pp_utah         77.6
FVCpp_utah          88.9
FEV1_FVC_utah       0.71
finalGold           -1.0
catEmph           normal
Name: 0, dtype: object

In [25]:
image_name = visit.build_image_name(state='EXP', recon='STD')
image_path = visit.image_file(variant='RAW', image_name=image_name)
image_nifti = nib.load(image_path)
image_nifti

In [31]:
def get_header_info(nifti):
    shape = np.asarray(nifti.header.get_data_shape())
    zooms = np.asarray(nifti.header.get_zooms())

    A = nifti.affine
    steps  = A[:3,:3].copy()
    origin = A[:3, 3].copy()
    axcodes = nib.orientations.aff2axcodes(A)

    center_ijk = (shape - 1) / 2
    center_xyz = voxel_to_world_coords(center_ijk, A)[0]

    # compute bbox of voxel centers from corner voxels
    i = [0, shape[0] - 1]
    j = [0, shape[1] - 1]
    k = [0, shape[2] - 1]
    I, J, K = np.meshgrid(i, j, k, indexing='ij')
    corners_ijk = np.stack([I, J, K], axis=-1).reshape(-1, 3)
    corners_xyz = voxel_to_world_coords(corners_ijk, A)
    
    bbox_min = corners_xyz.min(axis=0)
    bbox_max = corners_xyz.max(axis=0)
    lengths = (shape - 1) * zooms

    return {
        'shape': tuple(shape),
        'zooms': tuple(zooms),
        'axes_codes': axcodes,
        'origin': tuple(origin),
        'center': tuple(center_xyz),
        'steps': steps,
        'bbox_min': tuple(bbox_min),
        'bbox_max': tuple(bbox_max),
        'lengths': tuple(lengths)
    }
    
def voxel_to_world_coords(coords, affine):
    assert affine.shape == (4, 4)
    coords = np.atleast_2d(coords)
    homogeneous = np.c_[coords, np.ones(len(coords))]
    return (affine @ homogeneous.T).T[:,:3]

def world_to_voxel_coords(coords, affine):
    assert affine.shape == (4, 4)
    coords = np.atleast_2d(coords)
    homogeneous = np.c_[coords, np.ones(len(coords))]
    return np.linalg.solve(affine, homogeneous.T).T[:,:3]

def get_center(image):
    size = np.array(image.GetSize(), dtype=int)
    spacing = np.array(image.GetSpacing(), dtype=float)
    origin = np.array(image.GetOrigin(), dtype=float)
    D = np.array(image.GetDirection(), dtype=float).reshape(3, 3)
    return origin + D @ (spacing * (size - 1) / 2)

get_header_info(image_nifti)

{'shape': (512, 512, 537),
 'zooms': (0.71875, 0.71875, 0.5),
 'axes_codes': ('L', 'A', 'S'),
 'origin': (167.640625, -6.640625, -319.0),
 'center': (-16.0, 177.0, -185.0),
 'steps': array([[-0.71875, -0.     ,  0.     ],
        [ 0.     ,  0.71875,  0.     ],
        [ 0.     ,  0.     ,  0.5    ]]),
 'bbox_min': (-199.640625, -6.640625, -319.0),
 'bbox_max': (167.640625, 360.640625, -51.0),
 'lengths': (367.28125, 367.28125, 268.0)}

In [ ]:
%%time
visit.resample_images()

In [ ]:
%%time
visit.create_segmentation_masks()

In [22]:
%%time
visit.create_lung_regions_mask(
    lungs_kws={'connectivity': 1, 'max_components': 2},
    airways_kws={'connectivity': 1},
    vessels_kws={'connectivity': 1}
)

[16514P_EXP_STD_TEM_COPD] Creating lung regions mask
Filtering connected components: lungs
  Input mask has 6 components
    78 voxels were dropped (0.0036%)
  Output mask has 2 components
Filtering connected components: airways
  Input mask has 29 components
    134 voxels were dropped (0.8353%)
  Output mask has 1 components
Filtering connected components: vessels
  Input mask has 8071 components
    23907 voxels were dropped (12.5897%)
  Output mask has 63 components
Combining into lung regions mask
Mask has 1 components (1-connectivity)
Mask has 1 components (2-connectivity)
Mask has 1 components (3-connectivity)
CPU times: user 6.42 s, sys: 834 ms, total: 7.26 s
Wall time: 9.22 s


In [21]:
%%time
visit.create_anatomical_mesh(
    max_facet_distance=1.0,
    max_cell_circumradius=20.,
    seed=0
)[0]

[16514P_EXP_STD_TEM_COPD] Creating anatomical mesh
Removed 1 unused points
Final mesh:  <meshio mesh object>
  Number of points: 14792
  Number of cells:
    triangle: 52300
    tetra: 74664
  Cell data: label
Mesh has 1 components
[16514P_EXP_STD_TEM_COPD] Applying affine to mesh
CPU times: user 4.55 s, sys: 323 ms, total: 4.87 s
Wall time: 4.34 s


In [41]:
%%time
visit.create_displacement_field()

Compute fixed MIND features ... finished (0.26 s).
Stage 1/2
    search radius: 16
      cube length: 6
     quantisation: 2
     patch radius: 3
        transform: n
    Compute moving MIND features ... finished (0.24 s).
    1752 fixed keypoints extracted (0.20 s).
    Compute forward marginals ... finished (1.21 s).
    Compute symmetric backward marginals ... finished (1.22 s).
    Dense thin plate spline interpolation ... finished (0.55 s).

Stage 2/2
    search radius: 8
      cube length: 3
     quantisation: 1
     patch radius: 2
        transform: n
    Compute moving MIND features ... finished (0.21 s).
    3462 fixed keypoints extracted (0.19 s).
    Compute forward marginals ... finished (0.86 s).
    Compute symmetric backward marginals ... finished (0.94 s).
    Dense thin plate spline interpolation ... finished (1.09 s).

CPU times: user 23.8 s, sys: 1.77 s, total: 25.6 s
Wall time: 26.2 s
